# Projet Chatbot
## Emilie Longuevre / Léna Nock / Valentine Carteron

### 1. Importation des packages

In [1]:
import requests
from bs4 import BeautifulSoup, Tag, NavigableString
from urllib.error import HTTPError, URLError
from urllib.request import urlopen
import pickle
import pandas as pd

In [2]:
#site surlequel on réalise le webscrapping: Foire aux questions de la page Vinted
site = "https://www.vinted.fr/help"
# commande pour accéder aux données 
response = requests.get(site)

### 2. Récupérer toutes les urls des catégories du centre d'aide 
On récupère les urls des neuf catégories du centre d'aide.

In [3]:
urls_categorie1 = []
href = (BeautifulSoup(response.text)
          .select("html body div.site div.container div.faq div.faq__categories a.faq__category"))
for ref in href :
    urls_categorie1.append("https://www.vinted.fr" + ref.get('href'))
urls_categorie1

['https://www.vinted.fr/help/16-comment-ca-marche',
 'https://www.vinted.fr/help/4-vendre',
 'https://www.vinted.fr/help/5-acheter',
 'https://www.vinted.fr/help/313-envoi',
 'https://www.vinted.fr/help/418-paiements-et-transferts',
 'https://www.vinted.fr/help/15-confiance-et-securite',
 'https://www.vinted.fr/help/392-mon-compte-parametres',
 'https://www.vinted.fr/help/383-communaute',
 'https://www.vinted.fr/help/103-se-connecter']

On enregistre cette liste au format .txt pour conserver une trace de ces urls.

In [6]:
# with open('urls_categorie1.txt', 'w') as f:
#     for url in urls:
#         f.write(f'{url}\n')

# urls = open("urls_categorie1.txt", "r")
# urls = urls.read().split("\n")
# urls = urls[:-1]
# urls

Pour accéder aux réponses on doit passer par des sous-catégories de question. On va donc récupérer l'ensemble des urls menant aux réponses. On créé donc une fonction afin d'optimiser le code. Au maximum, on a trois sous catégories, donc on lance la fonction 'récupération_url' trois fois.

In [15]:
def recuperation_url(urls) :
    urls_categorie_sup = []
    n=0
    for url in urls : 
        n+=1
        # on précise l'url qu'on va explorer
        response = requests.get(url)
        # on sélectionne les liens présents dans ces pages
        href = (BeautifulSoup(response.text)
                  .select("html body div.site div.container div.faq a.c-cell__link"))
        # s'il y a une sous catégorie:
        if href != [] :
            for ref in href :
                # on ajoute l'url qui suit dans la liste des catégories supérieurs
                urls_categorie_sup.append("https://www.vinted.fr" + ref.get('href'))
        # s'il n'y a pas de sous catégories:
        else : 
            # on ajoute l'url actuel
            urls_categorie_sup.append(url)
        print(n)
    return urls_categorie_sup

In [9]:
urls_categorie2 = recuperation_url(urls_categorie1)

In [17]:
len(urls_categorie2)

52

In [17]:
# with open('urls_nv2.txt', 'w') as f:
#     for url in urls_nv_2:
#         f.write(f'{url}\n')

# urls_nv_2 = open("urls_nv2.txt", "r")
# urls_nv_2 = urls_nv_2.read().split("\n")
# urls_nv_2 = urls_nv_2[:-1]

In [21]:
urls_categorie3 = recuperation_url(urls_categorie2)
len(urls_categorie3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


216

In [24]:
# with open('urls_nv3.txt', 'w') as f:
#     for url in url_nv3:
#         f.write(f'{url}\n')

# urls_nv3 = open("urls_nv3.txt", "r")
# urls_nv3 = urls_nv3.read().split("\n")
# urls_nv3 = urls_nv3[:-1]

In [22]:
urls_categorie4 = recuperation_url(urls_categorie3)
len(urls_categorie4)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216


221

In [28]:
# with open('urls_nv4.txt', 'w') as f:
#     for url in url_nv4:
#         f.write(f'{url}\n')

# url_nv4 = open("urls_nv4.txt", "r")
# url_nv4 = url_nv4.read().split("\n")
# url_nv4 = url_nv4[:-1]

In [ ]:
urls_categorie4

### 3. Récupérer les réponses associés aux urls des questions

In [14]:
# demander valentine 
def ValidTag(monTag):
    if type(monTag) == Tag and monTag.has_attr("class"):
        if ' '.join(monTag.attrs["class"]) in ['tab-content', 'nav-tabs-horizontal', '“table-wrapper”']:
            return False
    return True

In [15]:
def getSelectedText(monTag):
    texte = []
    for child in monTag.children :
        if type(child) == NavigableString :
            if (child.string).strip() != '' and (child.string).strip() != ' ' :
                texte.append( " " + (child.string).strip() )
                
        ## Récupération des tableaux
        if type(child) == Tag and child.has_attr("class") and ' '.join(child.attrs["class"]) in ['“table-wrapper”']:
            tables = child.findAll('table')


            l = []
            nbr_valeur_ligne = []
            colonne = []
        
            # pour chaque ligne du tableau:
            for i,tr in enumerate(tables[0].findAll('tr')):
                # si la ligne contient un 'rowspan' c'est à dire une case pour plusieurs lignes
                if 'rowspan' in str(tr) : 
                    # récupère le numéro de cette ligne
                    valeur_depart = i 
                    # on ajoute à une liste l'ensemble des textes présents sur la ligne
                l.append(tr.text.replace('\n', '|#|'))
            # on récupère l'entête du tableau
            liste_entete = l[0].split('|#|')
            # on supprime les élément vides
            while '' in liste_entete : 
                liste_entete.remove('')
                
            a_change = []
            # on parcourt les lignes du tableaux ayant l'attribut 'td'
            for i,td in enumerate(tables[0].findAll('td')) :
                # si cet attribut contient un 'rowspan':
                if td.has_attr("rowspan") :
                    # on récupère le numéro de la colonne correspondant
                    num_colonne = i % len(liste_entete) 
                    # nombre de lignes concernées par cette exception
                    nbr_fois_lig = td.attrs['rowspan']
                    # valeur correspondante qui doit être recopiée
                    valeur_a_ajouter = td.text
                    # on récupère l'indice des lignes ayant une valeur manquante
                    a_change = [valeur_depart + o for o in range(2,int(nbr_fois_lig)+1)]

                    
            # création d'un compteur correspondant aux index des lignes        
            n = 1 
            # pour chaque ligne du tableau (sans le nom des colonnes)
            for mot in l[1::] : 
                n +=1
                # on découpe la ligne selon '|#|'
                # on obtient une liste contenant l'ensemble des mots de la ligne
                ligne = mot.split('|#|') 
                while '' in ligne : 
                    ligne.remove('')
                # si l'index de la ligne est présent dans la liste à changer
                if n in a_change: 
                    # on ajoute la valeur à ajouter au numéro de colonne correspondant
                    ligne[num_colonne:num_colonne] = [valeur_a_ajouter]
                # affichage final du tableau
                for i in range(0,len(liste_entete)) : 
                    texte.append( str(liste_entete[i]) +  " : " + str(ligne[i]) )
                texte.append( '------' )      
                
        
                
        if type(child) == Tag and ValidTag(child) and child.name != 'table':
            if getSelectedText(child).strip() != '' and getSelectedText(child).strip() != ' ' :
                texte.append( getSelectedText(child).strip() )
                
                
        if type(child) == Tag and child.name == 'table':
            tables = child.find('table')
        
            l = []
            nbr_valeur_ligne = []
            colonne = []
        
            # pour chaque ligne du tableau:
            for i,tr in enumerate([child][0].findAll('tr')):
                # si la ligne contient un 'rowspan' c'est à dire une case pour plusieurs lignes
                if 'rowspan' in str(tr) : 
                    # récupère le numéro de cette ligne
                    valeur_depart = i 
                    # on ajoute à une liste l'ensemble des textes présents sur la ligne
                l.append(tr.text.replace('\n', '|#|'))
            # on récupère l'entête du tableau
            liste_entete = l[0].split('|#|')
            # on supprime les élément vides
            while '' in liste_entete : 
                liste_entete.remove('')
                
            a_change = []
            # on parcourt les lignes du tableaux ayant l'attribut 'td'
            for i,td in enumerate([child][0].findAll('td')) :
                # si cet attribut contient un 'rowspan':
                if td.has_attr("rowspan") :
                    # on récupère le numéro de la colonne correspondant
                    num_colonne = i % len(liste_entete) 
                    # nombre de lignes concernées par cette exception
                    nbr_fois_lig = td.attrs['rowspan']
                    # valeur correspondante qui doit être recopiée
                    valeur_a_ajouter = td.text
                    # on récupère l'indice des lignes ayant une valeur manquante
                    a_change = [valeur_depart + o for o in range(2,int(nbr_fois_lig)+1)]

                    
            # création d'un compteur correspondant aux index des lignes        
            n = 1 
            # pour chaque ligne du tableau (sans le nom des colonnes)
            for mot in l[1::] : 
                n +=1
                # on découpe la ligne selon '|#|'
                # on obtient une liste contenant l'ensemble des mots de la ligne
                ligne = mot.split('|#|') 
                while '' in ligne : 
                    ligne.remove('')
                # si l'index de la ligne est présent dans la liste à changer
                if n in a_change: 
                    # on ajoute la valeur à ajouter au numéro de colonne correspondant
                    ligne[num_colonne:num_colonne] = [valeur_a_ajouter]
                # affichage final du tableau
                for i in range(0,len(liste_entete)) : 
                    texte.append( str(liste_entete[i]) +  " : " + str(ligne[i]) )
                texte.append( '------' )             
        
    texte_final = '\n'.join(texte)
    return texte_final

On peut donc désormais récupérer l'ensemble des textes qui correspondent aux réponses.

In [ ]:
liste_reponses = []
for url in urls_categorie4 :
    try :
        html = urlopen(url)
    except (HTTPError, URLError) as e :
        sys.exit(e)
    
    # On récupère les réponses dans la variable 'liste_reponses'
    # On remplace certains codes par leur signification
    bsObj = BeautifulSoup(html, "lxml")
    maDiv = bsObj.find("div", class_="site").find("div", class_="body-content").find("div", class_="faq__question-answer faq-content")
    texte = getSelectedText(maDiv).strip().replace('\xa0', ' ').replace('\u200b', ' ').replace('\u2192', ' ').replace('\u2194', ' ').replace('\u2028', ' ')
    liste_reponses.append(texte)
liste_reponses[0:3]

https://www.vinted.fr/help/26-vendre-etape-par-etape
https://www.vinted.fr/help/25-acheter-sur-vinted-toutes-les-etapes
https://www.vinted.fr/help/373-vendre-sur-vinted-c-est-gratuit
https://www.vinted.fr/help/356-comment-partager-mon-retour-d-experience-sur-vinted
https://www.vinted.fr/help/265-comment-utiliser-le-centre-d-aide
https://www.vinted.fr/help/375-ajouter-un-article-etape-par-etape
https://www.vinted.fr/help/26-vendre-etape-par-etape
https://www.vinted.fr/help/373-vendre-sur-vinted-c-est-gratuit
https://www.vinted.fr/help/52-qu-est-ce-que-je-peux-vendre-sur-vinted
https://www.vinted.fr/help/220-comment-echanger-un-article
https://www.vinted.fr/help/375-ajouter-un-article-etape-par-etape
https://www.vinted.fr/help/601-quelles-photos-dois-je-mettre-en-ligne-pour-mes-articles-de-marque
https://www.vinted.fr/help/48-comment-prendre-de-bonnes-photos
https://www.vinted.fr/help/49-comment-bien-decrire-mon-article
https://www.vinted.fr/help/50-comment-choisir-l-etat-de-mon-article


In [8]:
# with open('Reponses.txt', 'w') as f:
#     for texte in liste_textes:
#         f.write(f'{str(texte)}|#|')

In [ ]:
# ajouter le retour à la ligne

### 4. Récupérer toutes les questions

In [4]:
# On récupère le nom des neuf premières catégories 
question = (BeautifulSoup(response.text)
          .select("html body div.site div.container div.faq div.faq__categories  div.faq__category-title"))

liste_categories = []
for i in range(0,len(question)) :
    liste_categories.append(question[i].text.strip())
liste_categories

['Comment ça marche',
 'Vendre',
 'Acheter',
 'Envoi',
 'Paiements et transferts',
 'Confiance et sécurité',
 'Mon compte & Paramètres',
 'Communauté',
 'Se connecter']

On créé une fonction permettant de récupérer l'ensemble des chemins menant aux questions.

In [ ]:
def chemin_question(urls_cat, liste_prec):
    liste_chemin = []
    num=0
    # pour chaque url de la catégorie:
    for url in urls_cat : 
            response = requests.get(url)
            # on récupère le titre des sous catégories/questions
            question = (BeautifulSoup(response.text)
                  .select("html body div.site div.container div.faq div.c-cell__title"))
            # s'il y a des questions:
            if question != [] :
                # pour chaque numéro de question:
                for i in range(0,len(question)):
                    # on ajoute le chemin précédent + le titre de la sous catégorie/question
                    liste_chemin.append(liste_prec[num] + ' ' + question[i].text.strip())
            else:
                # sinon on garde le chemin actuel
                liste_chemin.append( liste_prec[num] )
            num+=1        
    return liste_chemin

In [ ]:
chemin1 = chemin_question(urls_categorie1, liste_categories)

In [ ]:
chemin2 = chemin_question(urls_categorie2, chemin1)

chemin3 = chemin_question(urls_categorie3, chemin2)

In [ ]:
liste_questions = chemin_question(urls_categorie4, chemin3)

### 5. Création d'un dataframe contenant les questions et les réponses 

In [ ]:
dataQR = pd.DataFrame({'question': chemin4, 'reponse': liste_textes})
dataQR['question'] = dataQR['question'].astype(str)
dataQR.head(10)

In [ ]:
# On vérifie combien de questions et de réponses on a.
print("Le nombre de questions est : ", len(dataQR['question']))
print("Le nombre de réponses est : ", len(dataQR['reponse']))

# afficher les 15 premières données
dataQR['question'].values[:15] 

On enregistre le dataframe obtenu ici pour pouvoir l'utiliser dans un nouveau notebook.

In [ ]:
dataQR.to_csv("dataQR.csv")